#### This notebook contains evaluations of the Bio-Bert and Roberta Model on test data.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, jaccard_score,classification_report 

import torch
import tensorflow as tf
from transformers import RobertaTokenizer,BertTokenizer,BertForSequenceClassification,BertModel,TFBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

!pip install jaro-winkler

2022-12-11 00:28:06.292091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:06.293114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:06.294223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:06.295054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:06.295843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-batch-800-809csv/training_batch_800_809.csv
/kaggle/input/bert-model-saved/config.json
/kaggle/input/bert-model-saved/tokenizer_config.json
/kaggle/input/bert-model-saved/pytorch_model.bin
/kaggle/input/bert-model-saved/special_tokens_map.json
/kaggle/input/pubmed-multi-label/roberta_results.csv
/kaggle/input/pubmed-multi-label/Classification_Report.csv
/kaggle/input/pubmed-multi-label/train_data_loader
/kaggle/input/pubmed-multi-label/__results__.html
/kaggle/input/pubmed-multi-label/test_data_loader
/kaggle/input/pubmed-multi-label/roberta_model
/kaggle/input/pubmed-multi-label/validation_data_loader
/kaggle/input/pubmed-multi-label/val_results.csv
/kaggle/input/pubmed-multi-label/__notebook__.ipynb
/kaggle/input/pubmed-multi-label/__output__.json
/kaggle/input/pubmed-multi-label/custom.css
/kaggle/input/pubmed-multi-label/Multi_label_Classification_Save/config.json
/kaggle/input/pubmed-multi-label/Multi_label_Classification_Save/tokenizer_config.json
/kaggle/i

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2022-12-11 00:28:34.719507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:34.720182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:34.720940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:34.721376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:28:34.722084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [5]:
dataset_Name = '/kaggle/input/pubmedtest/training_batch_810_819.csv'

df= pd.read_csv(dataset_Name)#.head(1000)

cols = list(df.columns)

# might have to change depending on categories 
mesh_Heading_categories = cols[6:]
num_labels = len(mesh_Heading_categories)

In [6]:
len(mesh_Heading_categories)

15

#### Bert Predictions 

In [7]:
# # df_train, df_test = train_test_split(df, random_state=32, test_size=0.20, shuffle=True)
df_test = df.copy()
df_test['one_hot_labels'] = list(df_test[mesh_Heading_categories].values)

test_labels = list(df_test.one_hot_labels.values)
Articles_test = list(df_test.AbstractText.values)
test_mesh_categories = list(df_test.columns[6:21])

# tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True)
# tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-model-saved/tokenizer_config.json', do_lower_case=True)
max_length = 128

test_encodings = tokenizer.batch_encode_plus(Articles_test,max_length=max_length,padding=True,truncation=True)
test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']


batch_size = 128
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels,)# test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_810_819_bert_data_loader')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)


In [25]:
num_labels=len(mesh_Heading_categories)
model = BertForSequenceClassification.from_pretrained("/kaggle/input/bert-model-saved/", num_labels=num_labels)

In [26]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]


In [11]:
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding


In [12]:
import jaro
from sklearn.metrics import hamming_loss, jaccard_score, classification_report, multilabel_confusion_matrix, f1_score, accuracy_score, log_loss 
from Levenshtein import distance as levenshtein_distance


def eval_test_report(true_bools,pred_bools, model_name):
    '''
    true_label: numpy array consisting of list (length 15 = num_labels) of actual labels [1,0,1,...,0]
    pred_label: numpy array consisting of list (length 15 = num_labels) of predicted labels [1,0,1,...,0]
    '''
    
    report_dict = {}
    report_dict['jaccard'] = 100.0*jaccard_score(true_bools,pred_bools, average="macro")
    report_dict['f1_score_micro'] = 100.0*f1_score(true_bools, pred_bools,average='micro')
    report_dict['f1_score_macro'] = 100.0*f1_score(true_bools, pred_bools,average='macro')
    report_dict['accuracy_score'] = 100.0*accuracy_score(true_bools, pred_bools)
    
    # lower the better
    report_dict['hamming_loss*'] = hamming_loss(true_bools, pred_bools)
    report_dict['log_loss*'] = log_loss(true_bools,pred_bools)
    
    # similarity by string matching
    pred_str = [''.join(list(map(str, (list(x))))) for x in list(pred_bools)]
    true_str = [''.join(list(map(str, (list(x))))) for x in list(true_bools)]

    sim = []
    i = 0
    while i < len(pred_str):
        sim.append(jaro.jaro_winkler_metric(true_str[i],pred_str[i]))
        i+=1
    
    report_dict['jaro_mean'] = 100.0*np.mean(sim)
    
    sim = []
    i = 0
    while i < len(pred_str):
        sim.append(levenshtein_distance(true_str[i],pred_str[i]))
        i+=1
    
    report_dict['lev_mean*'] = np.mean(sim)
    
#     print(classification_report(true_bools,pred_bools,target_names=test_mesh_categories)
    final = {model_name: report_dict}
    
    return pd.DataFrame(final)
    

In [14]:
# Final Report
report_bert = eval_test_report(true_bools,pred_bools,model_name = 'bio-bert')
report_bert.to_csv("bert_evaluation.csv")
report_bert

,bio-bert
accuracy_score,17.964407
f1_score_macro,76.402175
f1_score_micro,85.572135
hamming_loss*,0.111592
jaccard,64.170488
jaro_mean,92.031463
lev_mean*,6.597424
log_loss*,42.382964


In [27]:
## segment (20%) of test data from train set

data = pd.read_csv("/kaggle/input/training-batch-800-809csv/training_batch_800_809.csv")
df_train, df_test = train_test_split(data, random_state=32, test_size=0.20, shuffle=True)

df_test['one_hot_labels'] = list(df_test[mesh_Heading_categories].values)

test_labels = list(df_test.one_hot_labels.values)
Articles_test = list(df_test.AbstractText.values)
test_mesh_categories = list(df_test.columns[6:21])

# tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True)
# tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-model-saved/tokenizer_config.json', do_lower_case=True)
max_length = 128

test_encodings = tokenizer.batch_encode_plus(Articles_test,max_length=max_length,padding=True,truncation=True)
test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']


batch_size = 128
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels,)# test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
# torch.save(test_dataloader,'test_810_819_bert_data_loader')

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]

pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Final Report
report_bert_mini_test = eval_test_report(true_bools,pred_bools,model_name = 'bio-bert')
report_bert_mini_test

,bio-bert
accuracy_score,18.375586
f1_score_macro,76.443762
f1_score_micro,85.698372
hamming_loss*,0.110315
jaccard,64.270727
jaro_mean,92.142947
lev_mean*,6.525314
log_loss*,41.979689


#### Roberta Predictions

In [14]:
model = torch.load("/kaggle/input/pubmed-multi-label/roberta_model")

In [16]:
# # df_train, df_test = train_test_split(df, random_state=32, test_size=0.20, shuffle=True)
df_test = df.copy()
df_test['one_hot_labels'] = list(df_test[mesh_Heading_categories].values)

test_labels = list(df_test.one_hot_labels.values)
Articles_test = list(df_test.AbstractText.values)
test_mesh_categories = list(df_test.columns[6:21])

tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base', do_lower_case=False)
# tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True)

max_length = 128

test_encodings = tokenizer.batch_encode_plus(Articles_test,max_length=max_length,padding=True,truncation=True)
test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']


batch_size = 128
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels,)aa# test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_810_819_roberta_data_loader')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

In [17]:
# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]


In [18]:
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding


In [19]:
import jaro
from sklearn.metrics import hamming_loss, jaccard_score, classification_report, multilabel_confusion_matrix, f1_score, accuracy_score, log_loss 
from Levenshtein import distance as levenshtein_distance


def eval_test_report(true_bools,pred_bools, model_name):
    '''
    true_label: numpy array consisting of list (length 15 = num_labels) of actual labels [1,0,1,...,0]
    pred_label: numpy array consisting of list (length 15 = num_labels) of predicted labels [1,0,1,...,0]
    '''
    
    report_dict = {}
    report_dict['jaccard'] = 100.0*jaccard_score(true_bools,pred_bools, average="macro")
    report_dict['f1_score_micro'] = 100.0*f1_score(true_bools, pred_bools,average='micro')
    report_dict['f1_score_macro'] = 100.0*f1_score(true_bools, pred_bools,average='macro')
    report_dict['accuracy_score'] = 100.0*accuracy_score(true_bools, pred_bools)
    
    # lower the better
    report_dict['hamming_loss*'] = hamming_loss(true_bools, pred_bools)
    report_dict['log_loss*'] = log_loss(true_bools,pred_bools)
    
    # similarity by string matching
    pred_str = [''.join(list(map(str, (list(x))))) for x in list(pred_bools)]
    true_str = [''.join(list(map(str, (list(x))))) for x in list(true_bools)]

    sim = []
    i = 0
    while i < len(pred_str):
        sim.append(jaro.jaro_winkler_metric(true_str[i],pred_str[i]))
        i+=1
    
    report_dict['jaro_mean'] = 100.0*np.mean(sim)
    
    sim = []
    i = 0
    while i < len(pred_str):
        sim.append(levenshtein_distance(true_str[i],pred_str[i]))
        i+=1
    
    report_dict['lev_mean*'] = np.mean(sim)
    
#     print(classification_report(true_bools,pred_bools,target_names=test_mesh_categories)
    final = {model_name: report_dict}
    
    return pd.DataFrame(final)
    

In [20]:
# Final Report
report_roberta = eval_test_report(true_bools,pred_bools,model_name = 'roberta')
report_roberta

,roberta
accuracy_score,16.912740
f1_score_macro,75.320716
f1_score_micro,85.287306
hamming_loss*,0.114331
jaccard,63.077289
jaro_mean,91.849154
lev_mean*,6.756530
log_loss*,42.206369


In [21]:
final_report = pd.concat([report_bert, report_roberta], axis=1)

In [22]:
final_report

,bio-bert,roberta
accuracy_score,17.964407,16.912740
f1_score_macro,76.402175,75.320716
f1_score_micro,85.572135,85.287306
hamming_loss*,0.111592,0.114331
jaccard,64.170488,63.077289
jaro_mean,92.031463,91.849154
lev_mean*,6.597424,6.756530
log_loss*,42.382964,42.206369


In [23]:
final_report.to_csv("bert_roberta_results.csv")

In [24]:
## segment (20%) of test data from train set
test_dataloader = torch.load('/kaggle/input/pubmed-multi-label/test_data_loader')

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]

pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding


report_roberta_mini_test = eval_test_report(true_bools,pred_bools,model_name = 'roberta')
report_roberta_mini_test

,roberta
accuracy_score,0.062690
f1_score_macro,27.429926
f1_score_micro,46.018290
hamming_loss*,0.394721
jaccard,20.137369
jaro_mean,86.065294
lev_mean*,20.025728
log_loss*,125.190575


In [29]:
pd.concat([report_bert_mini_test, report_roberta_mini_test], axis=1).to_csv("bert_roberta_mini_test.csv")
pd.concat([report_bert_mini_test, report_roberta_mini_test], axis=1)

,bio-bert,roberta
accuracy_score,18.375586,0.062690
f1_score_macro,76.443762,27.429926
f1_score_micro,85.698372,46.018290
hamming_loss*,0.110315,0.394721
jaccard,64.270727,20.137369
jaro_mean,92.142947,86.065294
lev_mean*,6.525314,20.025728
log_loss*,41.979689,125.190575
